Render a simgle committee meeting page, part of the committees/dist pipelines which generate the static website at https://oknesset.org

## Change to the project root directory

Inside Docker the directory should be `/pipelines`

In [16]:
import os

if os.getcwd() != '/pipelines':
    os.chdir('..')

PIPELINES_ROOT_DIR = os.getcwd()

print(PIPELINES_ROOT_DIR)

/pipelines


## Load the source data

Can take a few minutes on the first run, depending on bandwidth

In [2]:
import os

if not all(map(os.path.exists, ['data/committees/kns_committee/datapackage.json',
                                'data/people/committees/meeting-attendees/datapackage.json',
                                'data/members/mk_individual/datapackage.json'])):
    !{'KNESSET_LOAD_FROM_URL=1 dpp run --concurrency 4 '\
                                      './committees/kns_committee,'\
                                      './people/committee-meeting-attendees,'\
                                      './members/mk_individual'}

## Run the build pipeline

This pipeline aggregates the relevant data and allows to filter for quicker development cycles.

You can uncomment and modify the filter step in [committees/dist/knesset.source-spec.yaml](https://github.com/hasadna/knesset-data-pipelines/blob/master/committees/dist/knesset.source-spec.yaml#L30) under the `build` pipeline

The build pipeline can take a few minutes to process for the first time

In [3]:
import os

if not os.path.exists('data/committees/dist/build_meetings/datapackage.json'):
    !{'dpp run --verbose ./committees/dist/build'}

## Load required source data to RAM

In [4]:
from datapackage_pipelines_knesset.committees.dist.meeting_context import get_meeting_context_data

meeting_context_data = get_meeting_context_data()

## Initialize the Jinja environment

In [5]:
from datapackage_pipelines_knesset.committees.dist.template_functions import get_jinja_env

jinja_env = get_jinja_env('committees/dist/templates')

## Get a committee meeting row for testing

In [45]:
from datapackage import Package
import yaml

build_meetings_package = Package('data/committees/dist/build_meetings/datapackage.json')
meeting_rows_generator = build_meetings_package.get_resource('kns_committeesession').iter(keyed=True)

meeting_rows_generator = (r for r in meeting_rows_generator if r['KnessetNum'] == 20)

# filter meetings which have more then 2 attended mks
meeting_rows_generator = (r for r in meeting_rows_generator if len(r['attended_mk_individual_ids']) > 2)

meeting_row = next(meeting_rows_generator)
print(yaml.dump(meeting_row, allow_unicode=True, default_flow_style=False))

BroadcastUrl: http://main.knesset.gov.il/Activity/committees/Pages/AllCommitteesBroadcast.aspx?TopicID=7433
CommitteeID: 922
CommitteeSessionID: 562716
FinishDate: null
KnessetNum: 20
LastUpdatedDate: 2015-04-19 09:04:43
Location: חדר הוועדה, באגף הוועדות (קדמה), קומה 3, חדר 3750
Note: null
Number: 1
SessionUrl: http://main.knesset.gov.il/Activity/committees/Pages/AllCommitteesAgenda.aspx?Tab=3&ItemID=562716
StartDate: 2015-04-01 14:00:00
TypeDesc: פתוחה
TypeID: 161
attended_mk_individual_ids:
- 768
- 230
- 876
- 911
- 92
committee_name: ועדת הכספים
download_crc32c: /1BF2g==
download_filename: files/23/2/8/286490.DOC
download_filesize: 71168
invitees:
- name: אתי בן יוסף
  role: מנהלת ועדת הכנסת, כנסת ישראל
legal_advisors:
- אייל לב ארי
manager:
- טמיר כהן
mks:
- זאב אלקין – יו"ר הוועדה המסדרת
- ניסן סלומינסקי – היו"ר
- אלי אלאלוף
- מיקי לוי
- איוב קרא
- עאידה תומא-סלימאן
parts_crc32c: W/pKBA==
parts_filesize: 11218
parts_parsed_filename: files/5/6/562716.csv
text_crc32c: t643Ig==
text

## Render the meeting page

In [46]:
import os
from datapackage_pipelines_knesset.committees.dist.template_functions import build_template
from datapackage_pipelines_knesset.committees.dist.committees_common import get_meeting_path
from datapackage_pipelines_knesset.committees.dist import meeting_context

# reload the meeting_context module in case you made some changes to it
from importlib import reload
reload(meeting_context)

meeting_html_file_base_path = get_meeting_path(meeting_row)

build_template(jinja_env,
               "committeemeeting_detail.html",
               meeting_context.get_meeting_context(meeting_row, meeting_context_data, use_data=False),
               meeting_html_file_base_path,
               output_root_dir='data/committees/dist/dist')


DEBUG   :Starting new HTTP connection (1): storage.googleapis.com:80
DEBUG   :http://storage.googleapis.com:80 "GET /knesset-data-pipelines/data/committees/meeting_protocols_parts/files/5/6/562716.csv HTTP/1.1" 200 11218
DEBUG   :Starting new HTTP connection (2): storage.googleapis.com:80
DEBUG   :http://storage.googleapis.com:80 "GET /knesset-data-pipelines/data/committees/meeting_protocols_parts/files/5/6/562716.csv HTTP/1.1" 200 11218
INFO    :Building template committeemeeting_detail.html to data/committees/dist/dist/meetings/5/6/562716.html


## Copy the static files needed for the frontend

In [8]:
!{'dpp run ./committees/dist/copy_static_files'}


./committees/dist/copy_static_files: WAITING FOR OUTPUT

./committees/dist/copy_static_files: WAITING FOR OUTPUT

./committees/dist/copy_static_files: SUCCESS, processed 0 rows
INFO    :RESULTS:
INFO    :SUCCESS: ./committees/dist/copy_static_files {}


## View the rendered html

To view the output, run the following command from a terminal on the knesset-data-pipelines project root:

```
( cd data/committees/dist/dist; python -m http.server )
```

Followin script prints the preview url and url to same meeting on production website

In [47]:
print('\n-- rendered meeting - localhost url --\n')
print(f'http://localhost:8000/{meeting_html_file_base_path}')
print('\n-- rendered meeting - production url --\n')
print(f'https://oknesset.org/{meeting_html_file_base_path}')


-- rendered meeting - localhost url --

http://localhost:8000/meetings/5/6/562716.html

-- rendered meeting - production url --

https://oknesset.org/meetings/5/6/562716.html
